In [2]:
import torch
import pandas as pd 

import json 
from math import sqrt

In [3]:
Z_99_PERCENT = 2.576

def ci(t): 
    return (
        Z_99_PERCENT * 
        (t.std() / sqrt(t.size(0)))
    ).item()

def to_dict(t):
    t = t.float().sort().values
    quartile = t.size(0) // 4
    trunc = t[quartile:-quartile]
    trunc_mean = trunc.mean().item()
    ci_range = ci(trunc)

    return {
        'mean': t.mean().item(),
        'max': t.max().item(),
        'min': t.min().item(),
        'std': t.std().item(),
        'trunc-mean': trunc_mean,
        'CI-low': trunc_mean - ci_range,
        'CI-high': trunc_mean + ci_range
    }

def get_results(s):
    data = torch.load(f'../results/{s}_eval.pt')
    lens = data['lens']; rews = data['rews']

    return {'rewards': to_dict(rews), 'episode lens': to_dict(lens)}


In [6]:
import pandas as pd 
def compare_one(n, dir):
    d_last = get_results(f'{dir}ppo_{n}N_0_last')
    #d_best = get_results(f'ppo_{n}N_0')

    last_r = d_last['rewards']
    last_r['n'] = n
    last_l = d_last['episode lens']
    last_l['n'] = n

    '''
    best_r = d_best['rewards']
    best_r['name'] = 'Best'
    best_l = d_best['episode lens']
    best_l['name'] = 'Best'
    '''
    
    return last_r, last_l

def eval(dir=''):
    rs, ls = zip(*[
        compare_one(n, dir)
        for n in [10,20,40]
    ])
    return pd.DataFrame(rs).transpose()#, pd.DataFrame(ls)


In [13]:
eval(dir='deg5e/')

,0,1,2
mean,225.469864,174.026810,161.114014
max,549.098022,574.052246,586.527100
min,153.599762,136.549698,142.350616
std,106.690361,79.759674,33.488491
trunc-mean,186.042252,150.146255,158.031876
CI-low,179.726615,149.260501,157.640253
CI-high,192.357888,151.032010,158.423498
n,10.000000,20.000000,40.000000


In [8]:
eval(dir='deg1e/')

,0,1,2
mean,232.157043,186.053757,160.912735
max,549.098022,574.052246,586.527100
min,153.799759,137.849670,143.100647
std,120.576317,87.046310,33.196182
trunc-mean,180.684418,157.616287,157.781677
CI-low,176.459731,155.530145,157.374338
CI-high,184.909105,159.702430,158.189016
n,10.000000,20.000000,40.000000


In [ ]:
eval(dir='og1e/')